In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
from os.path import isfile, join

%matplotlib inline

In [5]:
data_df = pd.DataFrame()
onlyfiles = [f for f in listdir('C:\Google_Drive\Инвестирование\ПИФ') if isfile(join('C:\Google_Drive\Инвестирование\ПИФ', f))]
for filename in onlyfiles:
    if filename[:6] != 'ШАБЛОН':
        init_df = pd.read_excel('C:\Google_Drive\Инвестирование\ПИФ\\'+filename, sheetname=1)
        init_df['Дата_2'] = pd.to_datetime(init_df['Дата_2'])
        init_df[filename[:len(filename)-5]] = init_df['Пок']
        if not('Дата_2' in data_df.columns):
            data_df['Дата_2'] = init_df['Дата_2']
            data_df[filename[:len(filename)-5]] = init_df[filename[:len(filename)-5]]
        else:
            data_df = pd.merge (data_df, init_df[['Дата_2', filename[:len(filename)-5]]], how='inner', on=['Дата_2'])
del data_df['Дата_2']
corr_df = data_df.corr('pearson')

In [6]:
rate_df = pd.read_excel('Rate.xlsx')
rate_df = rate_df.sort_values(by=['Итого2'], ascending=False).reset_index(drop = True)
rate_df['Dif'] = None
rate_df['Dif^1/64'] = None
rate_df['Dif_ln'] = None
rate_df['Итого3'] = None

In [7]:
for index in rate_df.index: #Переделать!!! заменить на простой итератор количества
    #print (rate_df.loc[index]['Название'])
    rate_df['Temp'] = None
    rate_df['Temp2'] = None
    rate_df['Temp3'] = None
    for pif in rate_df[pd.isnull(rate_df['Итого3'])].index:
        dif = 1
        for pif2 in rate_df[pd.notnull(rate_df['Итого3'])].index:
            dif = dif * (1-corr_df[rate_df['Название'][pif]][rate_df['Название'][pif2]])
            #print ('pif: ', pif, rate_df['Название'][pif], 'pif2: ', pif2, rate_df['Название'][pif2], 'corr: ', corr_df[rate_df['Название'][pif]][rate_df['Название'][pif2]])
        #print ('Итого2*dif:', rate_df['Итого2'][pif] * dif)
        rate_df.ix[pif,'Temp'] = rate_df['Итого2'][pif] * (dif**(1/64))
        rate_df.ix[pif,'Temp2'] = (dif)
        rate_df.ix[pif,'Temp3'] = (dif**(1/64))
    rate_df.ix[rate_df['Temp'].idxmax(),'Итого3'] = rate_df.ix[rate_df['Temp'].idxmax(),'Temp']
    rate_df.ix[rate_df['Temp'].idxmax(),'Dif'] = rate_df.ix[rate_df['Temp'].idxmax(),'Temp2']
    rate_df.ix[rate_df['Temp'].idxmax(),'Dif^1/64'] = rate_df.ix[rate_df['Temp'].idxmax(),'Temp3']
#    if index == 2:
#        break
del rate_df['Temp']
del rate_df['Temp2']
del rate_df['Temp3']

In [8]:
writer = pd.ExcelWriter('Итого3_plus.xlsx')
rate_df.to_excel(writer, 'Данные')
writer.save()